In [1]:
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

# example of random oversampling to balance the class distribution
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from matplotlib import pyplot
from tensorflow import keras


import tensorflow_addons as tfa
import tensorflow as tf

from scipy import stats

2023-02-23 14:15:56.240739: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-23 14:15:56.546737: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-23 14:15:57.189698: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/guilherme/cpu-tensorflow-marcelo/nvidia-smi/envs/tf/lib/
2023-02-23 14:15:57.191867: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7';

In [2]:
# configs
EPOCHS = 10
BATCH_SIZE = 32

TIME_SERIES_SIZE = 2   # Determines the window size. Ex (4,9)
TIME_STEP_SHIFT  = 1   # Determines specifies the number of steps to move the window forward at each iteration.

baseFolder = "../data_2019_processed/"

# selected features
inputFeatures = ["activity","location","day_of_week",
                 "light","phone_lock","proximity",
                 "sound","time_to_next_alarm", "minutes_day"]
outputClasses = ["awake","asleep"]
#outputClasses = ["class"]

In [3]:
# y_test     = Array with real values
# yhat_probs = Array with predicted values
def printMetrics(y_test,yhat_probs):
    # predict crisp classes for test set deprecated
    #yhat_classes = model.predict_classes(X_test, verbose=0)
    #yhat_classes = np.argmax(yhat_probs,axis=1)
    yhat_classes = yhat_probs.round()
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_test, yhat_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(y_test, yhat_classes)
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, yhat_classes)
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, yhat_classes)
    print('F1 score: %f' % f1)
    # kappa
    kappa = cohen_kappa_score(y_test, yhat_classes)
    print('Cohens kappa: %f' % kappa)
    # ROC AUC
    auc = roc_auc_score(y_test, yhat_probs)
    print('ROC AUC: %f' % auc)
    # confusion matrix
    print("\Confusion Matrix")
    matrix = confusion_matrix(y_test, yhat_classes)
    print(matrix)
    
    results = dict()
    results['accuracy'] = accuracy
    results['precision'] = precision
    results['recall'] = recall
    results['f1_score'] = f1
    results['cohen_kappa_score'] = kappa
    results['roc_auc_score'] = auc
    return results

def showGlobalMetrics(metrics):
    accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score = 0,0,0,0,0,0
    for metric in metrics:
        accuracy = accuracy + metric['accuracy']
        precision = precision + metric['precision']
        recall = recall + metric['recall']
        f1_score = f1_score + metric['f1_score']
        cohen_kappa_score = cohen_kappa_score + metric['cohen_kappa_score']
        roc_auc_score = roc_auc_score + metric['roc_auc_score']
        
    # mean
    size = len(metrics)
    print(size)
    accuracy = accuracy / size
    precision = precision / size
    recall = recall / size
    f1_score = f1_score / size
    cohen_kappa_score = cohen_kappa_score / size
    roc_auc_score = roc_auc_score / size
    
    #show:\
    print("accuracy: ",accuracy)
    print("precision: ",precision)
    print("recall: ",recall)
    print("f1_score: ",f1_score)
    print("cohen_kappa_score: ",cohen_kappa_score)
    print("roc_auc_score: ",roc_auc_score)
    
def transform_data_type(dataframe):
    
    # transform inputs
    for column in inputFeatures:
        dataframe[column] = dataframe[column].astype('float32')
    
    # transform outputs
    for column in outputClasses:
        dataframe[column] = dataframe[column].astype('float32')
    
    return dataframe

# one-hot encoding function
def transform_output_nominal_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data['awake']
    dataset['asleep'] = one_hot_encoded_data['asleep']
    
    return dataset

# one-hot encoding function
def transform_output_numerical_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data[0]
    dataset['asleep'] = one_hot_encoded_data[1]
    
    return dataset


def create_dataset_time_series_with_one_output(X, y, window_time_steps=1, shift_step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - window_time_steps, shift_step):
        v = X.iloc[i:(i + window_time_steps)].values
        labels = y.iloc[i: i + window_time_steps]
        Xs.append(v)        
        ys.append(stats.mode(labels)[0][0])
        
    if len(y.columns) == 1:
        return np.array(Xs), np.array(ys).reshape(-1, 1)
    else:
        return np.array(Xs), np.array(ys).reshape(-1, len(y.columns))

In [4]:
X_train = pd.read_csv(baseFolder+"train/allData-classification-numeric-normalized.csv")
X_test  = pd.read_csv(baseFolder+"test/allData-classification-numeric-normalized.csv")
#X_train = pd.read_csv(baseFolder+"train/allData-classification-numeric-normalized_balanced_undersample.csv")
#X_test  = pd.read_csv(baseFolder+"test/allData-classification-numeric-normalized_balanced_oversample.csv")

#AA = pd.read_csv(baseFolder+"allData-classification-numeric-normalized.csv")
#X_train, X_test = train_test_split(AA,test_size=0.25)

In [5]:
print(X_train.info())
X_train

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407452 entries, 0 to 407451
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            407452 non-null  float64
 1   location            407452 non-null  float64
 2   timestamp           407452 non-null  float64
 3   day_of_week         407452 non-null  float64
 4   light               407452 non-null  float64
 5   phone_lock          407452 non-null  float64
 6   proximity           407452 non-null  float64
 7   sound               407452 non-null  float64
 8   time_to_next_alarm  407452 non-null  float64
 9   minutes_day         407452 non-null  float64
 10  timestamp_text      407452 non-null  object 
 11  class               407452 non-null  object 
dtypes: float64(10), object(2)
memory usage: 37.3+ MB
None


,activity,location,timestamp,day_of_week,light,phone_lock,proximity,sound,time_to_next_alarm,minutes_day,timestamp_text,class
0,0.00,0.0,0.210603,0.0,0.000175,0.0,1.0,0.000000,0.982143,0.597637,2018-05-15 14:20:45,awake
1,0.00,0.0,0.210603,0.0,0.000175,0.0,1.0,0.000000,0.982143,0.597637,2018-05-15 14:20:45,awake
2,0.25,0.5,0.210603,0.0,0.000165,0.0,1.0,0.604408,0.982044,0.598332,2018-05-15 14:21:15,awake
3,0.25,0.5,0.210603,0.0,0.001449,0.0,1.0,0.604408,0.982044,0.598332,2018-05-15 14:21:45,awake
4,0.25,0.5,0.210603,0.0,0.000198,0.0,1.0,0.601849,0.981944,0.599027,2018-05-15 14:22:15,awake
...,...,...,...,...,...,...,...,...,...,...,...,...
407447,0.25,1.0,0.215387,0.0,0.000000,1.0,1.0,0.644370,0.992956,0.549687,2018-06-12 13:11:39,awake
407448,0.25,1.0,0.215387,0.0,0.000000,1.0,1.0,0.644370,0.992956,0.550382,2018-06-12 13:12:09,awake
407449,0.25,1.0,0.215387,0.0,0.000538,1.0,1.0,0.624127,0.992857,0.551077,2018-06-12 13:13:37,awake
407450,0.00,1.0,0.215387,0.0,0.000000,0.0,0.0,0.540295,0.992758,0.551772,2018-06-12 13:14:07,awake


In [6]:
# transform output to one_hot_encoding for the testing dataset
X_test = transform_output_nominal_class_into_one_hot_encoding(X_test)

# transform output to one_hot_encoding for the testing dataset
X_train = transform_output_nominal_class_into_one_hot_encoding(X_train)


# transforms the input data to float32
X_test = transform_data_type(X_test)

# transforms the input data to float32
X_train = transform_data_type(X_train)

In [7]:
# selects the data to train and test
X_train_data = pd.DataFrame(data=X_train,columns=inputFeatures)
y_train_data = pd.DataFrame(data=X_train,columns=outputClasses)
# selec test dataset (fixed to all)
X_test_data = pd.DataFrame(data=X_test,columns=inputFeatures)
y_test_data = pd.DataFrame(data=X_test,columns=outputClasses)

X_train_data, y_train_data = create_dataset_time_series_with_one_output(   #timestamp
    X_train_data, 
    y_train_data, 
    TIME_SERIES_SIZE, 
    TIME_STEP_SHIFT
)

X_test_data, y_test_data = create_dataset_time_series_with_one_output(    #timestamp
    X_test_data, 
    y_test_data, 
    TIME_SERIES_SIZE, 
    TIME_STEP_SHIFT
)


print("shape: ",X_train_data.shape, y_train_data.shape)
print("Size: ",X_test_data.shape,y_test_data.shape)       

shape:  (407450, 2, 9) (407450, 2)
Size:  (136284, 2, 9) (136284, 2)


In [8]:
# transtorm data to tensor slices
test_dataset_series = tf.data.Dataset.from_tensor_slices((X_test_data, y_test_data))
train_dataset_series = tf.data.Dataset.from_tensor_slices((X_train_data, y_train_data))

2023-02-23 14:18:36.746360: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-23 14:18:36.789689: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-23 14:18:36.790029: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-23 14:18:36.790931: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [9]:
train_dataset_series


<TensorSliceDataset element_spec=(TensorSpec(shape=(2, 9), dtype=tf.float32, name=None), TensorSpec(shape=(2,), dtype=tf.float32, name=None))>

In [10]:
#client_test_dataset.window(size=4, shift=1, stride=1, drop_remainder=True)
#test_dataset_series.batch(BATCH_SIZE) # usado no federated learning

In [11]:
# batch data size
test_dataset_series1 = test_dataset_series.batch(BATCH_SIZE)
train_dataset_series1 = train_dataset_series.batch(BATCH_SIZE)

train_dataset_series1

<BatchDataset element_spec=(TensorSpec(shape=(None, 2, 9), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>

In [12]:
print("input_shape=[", X_train_data.shape[1], X_train_data.shape[2],"]")
print("output shape:",len(outputClasses))

verbose, epochs, batch_size = 1, 2, BATCH_SIZE
#verbose, epochs, batch_size = 1, 3, 16
model = keras.Sequential()
model.add(LSTM(64,activation="tanh", 
          input_shape=[X_train_data.shape[1], X_train_data.shape[2]]))
#model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(len(outputClasses), activation='softmax'))#softmax,sigmoid
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy     
# fit network
model.fit(train_dataset_series1, epochs=epochs, verbose=verbose, batch_size=batch_size) #, batch_size=batch_size, validation_split=0.1
# evaluate model
#accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
# predict
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

input_shape=[ 2 9 ]
output shape: 2
Epoch 1/2


2023-02-23 14:18:45.975506: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201
2023-02-23 14:18:51.948780: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


12733/12733 [==============================] - 110s 8ms/step - loss: 0.4606 - categorical_accuracy: 0.7997
Epoch 2/2
4259/4259 [==============================] - 11s 3ms/step

awake
Accuracy: 0.811387
Precision: 0.811085
Recall: 0.999592
F1 score: 0.895526
Cohens kappa: 0.024572
ROC AUC: 0.736209
\Confusion Matrix
[[   411  25660]
 [    45 110168]]

asleep
Accuracy: 0.813133
Precision: 0.896930
Recall: 0.015835
F1 score: 0.031120
Cohens kappa: 0.024707
ROC AUC: 0.736682
\Confusion Matrix
[[110408     47]
 [ 25420    409]]

Global
2
accuracy:  0.8122596929940418
precision:  0.854007215782189
recall:  0.5077133069762128
f1_score:  0.46332297668580813
cohen_kappa_score:  0.024639547125285877
roc_auc_score:  0.7364456715052952


In [13]:
print("input_shape=[", X_train_data.shape[1], X_train_data.shape[2],"]")
print("output shape:",len(outputClasses))

verbose, epochs, batch_size = 1, 2, BATCH_SIZE
#verbose, epochs, batch_size = 1, 3, 16
model = keras.Sequential()
model.add(LSTM(64,activation="tanh", 
          input_shape=[X_train_data.shape[1], X_train_data.shape[2]]))
#model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(len(outputClasses), activation='softmax'))#softmax,sigmoid
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy     
# fit network
model.fit(train_dataset_series1, epochs=epochs, verbose=verbose) #, batch_size=batch_size, validation_split=0.1
# evaluate model
#accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
# predict
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

input_shape=[ 2 9 ]
output shape: 2
Epoch 1/2
12733/12733 [==============================] - 98s 8ms/step - loss: 0.4616 - categorical_accuracy: 0.7987
Epoch 2/2
4259/4259 [==============================] - 12s 3ms/step

awake
Accuracy: 0.811240
Precision: 0.810956
Recall: 0.999610
F1 score: 0.895454
Cohens kappa: 0.023264
ROC AUC: 0.737859
\Confusion Matrix
[[   389  25682]
 [    43 110170]]

asleep
Accuracy: 0.812986
Precision: 0.895833
Recall: 0.014983
F1 score: 0.029473
Cohens kappa: 0.023384
ROC AUC: 0.738361
\Confusion Matrix
[[110410     45]
 [ 25442    387]]

Global
2
accuracy:  0.8121129406239911
precision:  0.8533946868651179
recall:  0.507296502426824
f1_score:  0.4624639083172795
cohen_kappa_score:  0.023323821332342065
roc_auc_score:  0.7381099422353801


In [14]:
print("input_shape=[", X_train_data.shape[1], X_train_data.shape[2],"]")
print("output shape:",len(outputClasses))

verbose, epochs, batch_size = 1, 2, BATCH_SIZE
#verbose, epochs, batch_size = 1, 3, 16
model = keras.Sequential()
model.add(LSTM(64,activation="tanh", 
          input_shape=[X_train_data.shape[1], X_train_data.shape[2]]))
#model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(len(outputClasses), activation='softmax'))#softmax,sigmoid
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy     
# fit network
#model.fit(train_dataset_series1, epochs=epochs, verbose=verbose) #, batch_size=batch_size, validation_split=0.1
model.fit(X_train_data, y_train_data, epochs=epochs, verbose=verbose, batch_size=batch_size) #, batch_size=batch_size, validation_split=0.1
# evaluate model
#accuracy = model.evaluate(X_test_data, y_test_data) # , batch_size=batch_size, verbose=0
#accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
# predict
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

input_shape=[ 2 9 ]
output shape: 2
Epoch 1/2
12733/12733 [==============================] - 101s 8ms/step - loss: 0.4373 - categorical_accuracy: 0.8102
Epoch 2/2
4259/4259 [==============================] - 11s 2ms/step

awake
Accuracy: 0.834581
Precision: 0.854720
Recall: 0.958344
F1 score: 0.903571
Cohens kappa: 0.335330
ROC AUC: 0.810961
\Confusion Matrix
[[  8118  17953]
 [  4591 105622]]

asleep
Accuracy: 0.836107
Precision: 0.637422
Recall: 0.313640
F1 score: 0.420416
Cohens kappa: 0.337617
ROC AUC: 0.811643
\Confusion Matrix
[[105847   4608]
 [ 17728   8101]]

Global
2
accuracy:  0.8353438408030289
precision:  0.7460710524720184
recall:  0.6359920021290034
f1_score:  0.661993484490602
cohen_kappa_score:  0.33647350925450514
roc_auc_score:  0.8113022054130092


In [15]:
print("input_shape=[", X_train_data.shape[1], X_train_data.shape[2],"]")
print("output shape:",len(outputClasses))

verbose, epochs, batch_size = 1, 2, 16
#verbose, epochs, batch_size = 1, 3, 16
model = keras.Sequential()
model.add(LSTM(64,activation="tanh", 
          input_shape=[X_train_data.shape[1], X_train_data.shape[2]]))
#model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(len(outputClasses), activation='softmax'))#softmax,sigmoid
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy     
# fit network
#model.fit(train_dataset_series1, epochs=epochs, verbose=verbose) #, batch_size=batch_size, validation_split=0.1
model.fit(X_train_data, y_train_data, epochs=epochs, verbose=verbose, batch_size=batch_size) #, batch_size=batch_size, validation_split=0.1
# evaluate model
#accuracy = model.evaluate(X_test_data, y_test_data) # , batch_size=batch_size, verbose=0
#accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
# predict
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

input_shape=[ 2 9 ]
output shape: 2
Epoch 1/2
25466/25466 [==============================] - 201s 8ms/step - loss: 0.4338 - categorical_accuracy: 0.8114
Epoch 2/2
4259/4259 [==============================] - 11s 2ms/step

awake
Accuracy: 0.837897
Precision: 0.881519
Recall: 0.923702
F1 score: 0.902118
Cohens kappa: 0.432244
ROC AUC: 0.808352
\Confusion Matrix
[[ 12388  13683]
 [  8409 101804]]

asleep
Accuracy: 0.839174
Precision: 0.594028
Recall: 0.478300
F1 score: 0.529919
Cohens kappa: 0.434271
ROC AUC: 0.808918
\Confusion Matrix
[[102012   8443]
 [ 13475  12354]]

Global
2
accuracy:  0.8385357048516333
precision:  0.7377735584145093
recall:  0.7010009356556297
f1_score:  0.7160183924566
cohen_kappa_score:  0.4332579370118738
roc_auc_score:  0.8086349563079785


In [16]:
print("input_shape=[", X_train_data.shape[1], X_train_data.shape[2],"]")
print("output shape:",len(outputClasses))

verbose, epochs, batch_size = 1, 5, BATCH_SIZE
#verbose, epochs, batch_size = 1, 3, 16
model = keras.Sequential()
model.add(LSTM(64,activation="tanh", 
          input_shape=[X_train_data.shape[1], X_train_data.shape[2]]))
#model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(len(outputClasses), activation='softmax'))#softmax,sigmoid
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy     
# fit network
#model.fit(train_dataset_series1, epochs=epochs, verbose=verbose) #, batch_size=batch_size, validation_split=0.1
model.fit(X_train_data, y_train_data, epochs=epochs, verbose=verbose, batch_size=batch_size) #, batch_size=batch_size, validation_split=0.1
# evaluate model
#accuracy = model.evaluate(X_test_data, y_test_data) # , batch_size=batch_size, verbose=0
#accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
# predict
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

input_shape=[ 2 9 ]
output shape: 2
Epoch 1/5
12733/12733 [==============================] - 104s 8ms/step - loss: 0.4383 - categorical_accuracy: 0.8100
Epoch 2/5
12733/12733 [==============================] - 108s 8ms/step - loss: 0.4176 - categorical_accuracy: 0.8177
Epoch 3/5
12733/12733 [==============================] - 100s 8ms/step - loss: 0.4125 - categorical_accuracy: 0.8191
Epoch 4/5
12733/12733 [==============================] - 95s 7ms/step - loss: 0.4080 - categorical_accuracy: 0.8202
Epoch 5/5
4259/4259 [==============================] - 10s 2ms/step

awake
Accuracy: 0.833341
Precision: 0.874642
Recall: 0.926742
F1 score: 0.899939
Cohens kappa: 0.404078
ROC AUC: 0.800767
\Confusion Matrix
[[ 11432  14639]
 [  8074 102139]]

asleep
Accuracy: 0.834559
Precision: 0.584128
Recall: 0.441132
F1 score: 0.502658
Cohens kappa: 0.405740
ROC AUC: 0.801354
\Confusion Matrix
[[102343   8112]
 [ 14435  11394]]

Global
2
accuracy:  0.8339496932875465
precision:  0.7293852223285127
recal

In [17]:
print("input_shape=[", X_train_data.shape[1], X_train_data.shape[2],"]")
print("output shape:",len(outputClasses))

verbose, epochs, batch_size = 1, 8, BATCH_SIZE
#verbose, epochs, batch_size = 1, 3, 16
model = keras.Sequential()
model.add(LSTM(64,activation="tanh", 
          input_shape=[X_train_data.shape[1], X_train_data.shape[2]]))
#model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(len(outputClasses), activation='softmax'))#softmax,sigmoid
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy     
# fit network
#model.fit(train_dataset_series1, epochs=epochs, verbose=verbose) #, batch_size=batch_size, validation_split=0.1
model.fit(X_train_data, y_train_data, epochs=epochs, verbose=verbose, batch_size=batch_size) #, batch_size=batch_size, validation_split=0.1
# evaluate model
#accuracy = model.evaluate(X_test_data, y_test_data) # , batch_size=batch_size, verbose=0
#accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
# predict
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

input_shape=[ 2 9 ]
output shape: 2
Epoch 1/8
12733/12733 [==============================] - 97s 7ms/step - loss: 0.4361 - categorical_accuracy: 0.8112
Epoch 2/8
12733/12733 [==============================] - 95s 7ms/step - loss: 0.4176 - categorical_accuracy: 0.8172
Epoch 3/8
12733/12733 [==============================] - 94s 7ms/step - loss: 0.4136 - categorical_accuracy: 0.8185
Epoch 4/8
12733/12733 [==============================] - 93s 7ms/step - loss: 0.4097 - categorical_accuracy: 0.8201
Epoch 5/8
12733/12733 [==============================] - 99s 8ms/step - loss: 0.4057 - categorical_accuracy: 0.8211
Epoch 6/8
12733/12733 [==============================] - 103s 8ms/step - loss: 0.4024 - categorical_accuracy: 0.8220
Epoch 7/8
12733/12733 [==============================] - 102s 8ms/step - loss: 0.3994 - categorical_accuracy: 0.8236
Epoch 8/8
4259/4259 [==============================] - 10s 2ms/step

awake
Accuracy: 0.825027
Precision: 0.843990
Recall: 0.961338
F1 score: 0.898850


In [18]:
print("input_shape=[", X_train_data.shape[1], X_train_data.shape[2],"]")
print("output shape:",len(outputClasses))

verbose, epochs, batch_size = 1, 2, BATCH_SIZE
#verbose, epochs, batch_size = 1, 3, 16
model = keras.Sequential()
model.add(LSTM(128,activation="tanh", 
          input_shape=[X_train_data.shape[1], X_train_data.shape[2]]))
#model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(len(outputClasses), activation='softmax'))#softmax,sigmoid
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy     
# fit network
#model.fit(train_dataset_series1, epochs=epochs, verbose=verbose) #, batch_size=batch_size, validation_split=0.1
model.fit(X_train_data, y_train_data, epochs=epochs, verbose=verbose, batch_size=batch_size) #, batch_size=batch_size, validation_split=0.1
# evaluate model
#accuracy = model.evaluate(X_test_data, y_test_data) # , batch_size=batch_size, verbose=0
#accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
# predict
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

input_shape=[ 2 9 ]
output shape: 2
Epoch 1/2
12733/12733 [==============================] - 102s 8ms/step - loss: 0.4344 - categorical_accuracy: 0.8118
Epoch 2/2
4259/4259 [==============================] - 10s 2ms/step

awake
Accuracy: 0.837450
Precision: 0.867720
Recall: 0.942711
F1 score: 0.903662
Cohens kappa: 0.389498
ROC AUC: 0.801845
\Confusion Matrix
[[ 10232  15839]
 [  6314 103899]]

asleep
Accuracy: 0.838815
Precision: 0.616705
Recall: 0.395060
F1 score: 0.481605
Cohens kappa: 0.391551
ROC AUC: 0.802580
\Confusion Matrix
[[104113   6342]
 [ 15625  10204]]

Global
2
accuracy:  0.8381321358339937
precision:  0.7422122323735725
recall:  0.6688853744762389
f1_score:  0.692633402147532
cohen_kappa_score:  0.39052467947206976
roc_auc_score:  0.8022125321575377


In [19]:
print("input_shape=[", X_train_data.shape[1], X_train_data.shape[2],"]")
print("output shape:",len(outputClasses))

verbose, epochs, batch_size = 1, 10, BATCH_SIZE
#verbose, epochs, batch_size = 1, 3, 16
model = keras.Sequential()
model.add(LSTM(128,activation="tanh", 
          input_shape=[X_train_data.shape[1], X_train_data.shape[2]]))
#model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(len(outputClasses), activation='softmax'))#softmax,sigmoid
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy     
# fit network
model.fit(train_dataset_series1, epochs=epochs, verbose=verbose) #, batch_size=batch_size, validation_split=0.1
# evaluate model
#accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
# predict
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

input_shape=[ 2 9 ]
output shape: 2
Epoch 1/10
12733/12733 [==============================] - 102s 8ms/step - loss: 0.4598 - categorical_accuracy: 0.8022
Epoch 2/10
12733/12733 [==============================] - 95s 7ms/step - loss: 0.4425 - categorical_accuracy: 0.8091
Epoch 3/10
12733/12733 [==============================] - 105s 8ms/step - loss: 0.4332 - categorical_accuracy: 0.8134
Epoch 4/10
12733/12733 [==============================] - 100s 8ms/step - loss: 0.4240 - categorical_accuracy: 0.8184
Epoch 5/10
12733/12733 [==============================] - 100s 8ms/step - loss: 0.4192 - categorical_accuracy: 0.8206
Epoch 6/10
12733/12733 [==============================] - 95s 7ms/step - loss: 0.4171 - categorical_accuracy: 0.8217
Epoch 7/10
12733/12733 [==============================] - 94s 7ms/step - loss: 0.4128 - categorical_accuracy: 0.8233
Epoch 8/10
12733/12733 [==============================] - 95s 7ms/step - loss: 0.4088 - categorical_accuracy: 0.8246
Epoch 9/10
12733/12733 [

In [20]:
print("input_shape=[", X_train_data.shape[1], X_train_data.shape[2],"]")
print("output shape:",len(outputClasses))

verbose, epochs, batch_size = 1, 50, BATCH_SIZE
#verbose, epochs, batch_size = 1, 3, 16
model = keras.Sequential()
model.add(LSTM(128,activation="tanh", 
          input_shape=[X_train_data.shape[1], X_train_data.shape[2]]))
#model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(len(outputClasses), activation='softmax'))#softmax,sigmoid
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy     
# fit network
model.fit(train_dataset_series1, epochs=epochs, verbose=verbose) #, batch_size=batch_size, validation_split=0.1
# evaluate model
#accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
# predict
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

input_shape=[ 2 9 ]
output shape: 2
Epoch 1/50
12733/12733 [==============================] - 103s 8ms/step - loss: 0.4590 - categorical_accuracy: 0.8030
Epoch 2/50
12733/12733 [==============================] - 104s 8ms/step - loss: 0.4434 - categorical_accuracy: 0.8100
Epoch 3/50
12733/12733 [==============================] - 104s 8ms/step - loss: 0.4293 - categorical_accuracy: 0.8150
Epoch 4/50
12733/12733 [==============================] - 101s 8ms/step - loss: 0.4219 - categorical_accuracy: 0.8199
Epoch 5/50
12733/12733 [==============================] - 103s 8ms/step - loss: 0.4170 - categorical_accuracy: 0.8221
Epoch 6/50
12733/12733 [==============================] - 103s 8ms/step - loss: 0.4119 - categorical_accuracy: 0.8238
Epoch 7/50
12733/12733 [==============================] - 102s 8ms/step - loss: 0.4077 - categorical_accuracy: 0.8251
Epoch 8/50
12733/12733 [==============================] - 103s 8ms/step - loss: 0.4060 - categorical_accuracy: 0.8257
Epoch 9/50
12733/127